In [ ]:
# Mount YOUR google drive. You'll need to "Add shortcut to Drive" for our shared folder for it to show up here.
# Use the URL shown below in the output to authorize this Colab session to access you GDrive
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [1]:
# Import needed packages
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats as sts
from scipy.stats import ttest_ind
from numpy.random import default_rng
from plotnine import *
import pandas as pd

import pandas as pd
import geopandas as gpd
import geopy
import geopy.distance
from shapely.geometry import Point, Polygon
import plotly.express as px

rng = default_rng(13)

In [ ]:
# Download the files associated with the GTA_Census project(.shp, .shx, .dbf, .cpg, .proj).
# Load these files onto the notebook.

# Read the shapefile as a dataframe object
df_census = gpd.read_file("spatial correlation\GTA_Census.shp")
df_census.crs = 'EPSG:3347'
df_census['CTUID'] = df_census['CTUID'].astype(str)

'''
These data represent census tracts (subdivisions of a city) in the
Greater Toronto Area (GTA). For each census tract, we have data on the *median*
household income in 2015 (HInc2015), in 2020 (HInc2020), and the 'delta'
between 2020 and 2015 (DHInc) - positive values represent increases in the
median household income in that census tract.
'''

df_census.head(3)

,CTUID,DGUID,HInc2020,HInc2015,DHInc,geometry
0,5350420.13,2021S05075350420.13,75500,81000,-5500,"POLYGON ((7218613.794 949619.114, 7218679.703 ..."
1,5350422.03,2021S05075350422.03,111000,128000,-17000,"POLYGON ((7219855.811 954804.291, 7219916.443 ..."
2,5350422.04,2021S05075350422.04,112000,119000,-7000,"POLYGON ((7217958.14 954705.514, 7217957.497 9..."


In [ ]:
# Transform into a geodataframe
gdf_census = gpd.GeoDataFrame(df_census)

# Shape file is a different CRS, change to lon/lat GPS co-ordinates
gdf_census = gdf_census.to_crs("WGS84") #==EPSG:4326
gdf_census

,CTUID,DGUID,HInc2020,HInc2015,DHInc,geometry
0,5350420.13,2021S05075350420.13,75500,81000,-5500,"POLYGON ((-79.40757 43.84423, -79.40717 43.842..."
1,5350422.03,2021S05075350422.03,111000,128000,-17000,"POLYGON ((-79.38047 43.88684, -79.38012 43.885..."
2,5350422.04,2021S05075350422.04,112000,119000,-7000,"POLYGON ((-79.40335 43.88924, -79.40336 43.889..."
3,5350422.05,2021S05075350422.05,108000,120000,-12000,"POLYGON ((-79.41589 43.89098, -79.41584 43.890..."
4,5350422.06,2021S05075350422.06,54000,63200,-9200,"POLYGON ((-79.43584 43.87158, -79.43726 43.871..."
...,...,...,...,...,...,...
1213,5350805.27,2021S05075350805.27,115000,136000,-21000,"POLYGON ((-79.02541 43.88692, -79.02605 43.886..."
1214,5350593.01,2021S05075350593.01,115000,124000,-9000,"POLYGON ((-79.97021 44.02639, -79.96982 44.024..."
1215,5350593.02,2021S05075350593.02,118000,124000,-6000,"POLYGON ((-79.98788 44.02693, -79.98755 44.025..."
1216,5350593.03,2021S05075350593.03,141000,142000,-1000,"POLYGON ((-80.10673 43.95614, -80.10564 43.951..."




---



## <font color='orange'> Task 0: Find all Census tracts/polygons that are neighbours to each other</font>

I will consider that two Census tracts are neighbours only if they share a vertex or boundary line.

It meams we consider Queen neighbourhood!

In [ ]:
# This code can help you find the neighbours of a polygon
## in this case, the first polygon in the dataframe, gdf_census.loc[0,'geometry']
## The id for this polygon is gdf_census.loc[0,'CTUID'], which results in 5350420.13


#To find which polygons are neighbours of each, we will seek those that are NOT neighbours first
##This line gdf_census.geometry.disjoint(gdf_census.loc[0,'geometry']) finds all polygons disjoint
  # of the first polygon in the dataframe (gdf_census.loc[0,'geometry'])
  # An object is said to be disjoint to other if its boundary and interior
  # does not intersect at all with those of the other.
  # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.disjoint.html


#The tilde (~) is equivalent to 'NOT', hence, ~gdf_census.geometry.disjoint(gdf_census.loc[0,'geometry'])
  # helps us find the polygons that do NOT meet the criteria of being disjoint of (gdf_census.loc[0,'geometry'])

#The last bit selects the name of the polygon (CTUID) and save to a list (.tolist())
neighbours = gdf_census[~gdf_census.geometry.disjoint(gdf_census.loc[0,'geometry'])].CTUID.tolist()
print('The neighbours of polygon',df_census.loc[0,'CTUID'],'are:')
neighbours

# Hint: the code above will include the polygon in its own neighbours list because a polygon is not disjoint of itself.
# ***You will need to fix that.***

The neighbours of polygon 5350420.13 are:


['5350420.13',
 '5350420.14',
 '5350420.15',
 '5350402.07',
 '5350402.08',
 '5350421.01',
 '5350402.09',
 '5350420.05',
 '5350420.06']

In [ ]:
# Task: For *all* polygon in the shapefile, identify its neighbours and
##  store the list of neighbours as a new column named 'neighbours' in the shapefile

# The example below is doing this for one polygon
gdf_census_example = gdf_census.head(1).copy()
gdf_census_example['neighbours'] = [neighbours]
gdf_census_example

,CTUID,DGUID,HInc2020,HInc2015,DHInc,geometry,neighbours
0,5350420.13,2021S05075350420.13,75500,81000,-5500,"POLYGON ((-79.40757 43.84423, -79.40717 43.842...","[5350420.13, 5350420.14, 5350420.15, 5350402.0..."


In [ ]:
##  store the list of neighbours as a new column named 'neighbours' in the shapefile
gdf_census['neighbours'] = gdf_census.apply(lambda row: gdf_census[~gdf_census.geometry.disjoint(row.geometry)].CTUID.tolist(), axis=1)
#to remove self-referene, otherwise it considered itself :)
gdf_census['neighbours'] = gdf_census.apply(lambda row: [ctuid for ctuid in row.neighbours if ctuid != row.CTUID], axis=1)

In [ ]:
gdf_census.head(3)

,CTUID,DGUID,HInc2020,HInc2015,DHInc,geometry,neighbours
0,5350420.13,2021S05075350420.13,75500,81000,-5500,"POLYGON ((-79.40757 43.84423, -79.40717 43.842...","[5350420.14, 5350420.15, 5350402.07, 5350402.0..."
1,5350422.03,2021S05075350422.03,111000,128000,-17000,"POLYGON ((-79.38047 43.88684, -79.38012 43.885...","[5350422.04, 5350422.05, 5350403.04, 5350403.1..."
2,5350422.04,2021S05075350422.04,112000,119000,-7000,"POLYGON ((-79.40335 43.88924, -79.40336 43.889...","[5350422.03, 5350422.05, 5350420.10]"


<font color='orange'> Plot the neighbours of one Census tract/polygon. You can use this to check your solution. </font>

In [ ]:
# To check your answer, we will plot the neighbours of the **first** Census tract/polygon
# Get hold of the neighbours
list_of_neighbours = gdf_census.loc[0,'neighbours']


# Slice the geo dataframe to only include polygons that are in the list of neighbours of polygon 0
gdf_new = gdf_census[gdf_census['CTUID'].isin(list_of_neighbours)].reset_index(drop=True)


# Plot
fig = px.choropleth_mapbox(
    gdf_new,
    geojson=gdf_new.geometry,
    locations=gdf_new.index,
    color="HInc2020",
    labels={'HInc2020':'Household Income in 2020'},
    color_continuous_scale="Sunset",
    center=dict(lat=gdf_new.loc[0,'geometry'].centroid.y, lon=gdf_new.loc[0,'geometry'].centroid.x),
    mapbox_style="open-street-map",
    opacity=0.9,
    zoom=11,
    height=600,
    width=800
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()



---



## <font color='orange'> Task 1: Calculate the Global Moran's I and Geary's C for the 2020 Income for the simplified case above [5 Points].</font>

<font color='orange'> This task we will work only with the polygons that are neighbours to the first Census tract/polygon. </font>


In [ ]:
# Slice the geo dataframe to only include polygons that are in the list of neighbours of polygon 0
gdf_new = gdf_census[gdf_census['CTUID'].isin(list_of_neighbours)].reset_index(drop=True)
gdf_new
# This should contain 8 polygons only

,CTUID,DGUID,HInc2020,HInc2015,DHInc,geometry,neighbours
0,5350420.14,2021S05075350420.14,75500,75500,0,"POLYGON ((-79.42467 43.85062, -79.42455 43.850...","[5350420.13, 5350420.15, 5350421.01, 5350420.0..."
1,5350420.15,2021S05075350420.15,101000,114000,-13000,"POLYGON ((-79.40897 43.8494, -79.40863 43.8480...","[5350420.13, 5350420.14, 5350420.03, 5350420.0..."
2,5350402.07,2021S05075350402.07,103000,110000,-7000,"POLYGON ((-79.40292 43.8259, -79.40288 43.8257...","[5350420.13, 5350402.08, 5350402.09, 5350420.05]"
3,5350402.08,2021S05075350402.08,100000,105000,-5000,"POLYGON ((-79.40201 43.83866, -79.40196 43.838...","[5350420.13, 5350402.12, 5350402.13, 5350402.0..."
4,5350421.01,2021S05075350421.01,105000,112000,-7000,"POLYGON ((-79.45373 43.82949, -79.45414 43.829...","[5350420.13, 5350411.29, 5350411.19, 5350420.1..."
5,5350402.09,2021S05075350402.09,84000,92000,-8000,"POLYGON ((-79.40286 43.82569, -79.40269 43.825...","[5350420.13, 5350402.03, 5350402.07, 5350402.0..."
6,5350420.05,2021S05075350420.05,71500,79500,-8000,"POLYGON ((-79.39284 43.84197, -79.39429 43.841...","[5350420.13, 5350403.04, 5350402.12, 5350420.1..."
7,5350420.06,2021S05075350420.06,114000,127000,-13000,"POLYGON ((-79.40801 43.84819, -79.40863 43.848...","[5350420.13, 5350420.15, 5350420.03, 5350420.0..."


In [ ]:
# Suggested steps:
## 1. For each polygon, create an adjacency list, lets call it adj_list_for_CT_i, ##my insight: only consider 8 poly, instead of 1218 polygone, make it small
### You can build this list by finding if Census tract's i 'CTUID' in list of 'neighbours' for Census tract j

## 2. Build a list of adjacency list, say master_list
### Using a loop, you can append adj_list for Census Tract i to the master_list
### master_list.append(adj_list_for_CT_i)

## 3. You can transform a list of lists to a numpy array using np.asarray(master_list)
### This will be your (raw) weights matrix (W)

## 4. If you want to normalize W to get standard weights, you can use
### W_std = np.nan_to_num(W / row_sums[:,np.newaxis],0)
### This code will take care of cases when there are no neighbours (otherwise we will have division by zero)

## 5. The GISA results you will obtain are not meaningful, as you only have 8 points, it is
###   not easy to identify a correlation pattern

## 6. Since you have few points, you can verify your results using an excel spreadsheet as the one shared in LEARN.

'''
Variations of steps 1-4 will be used in following HW. Make sure you understand these steps.
'''
# step1 and 2
master_list = []
for i, row_i in gdf_new.iterrows():
    adj_list_for_CT_i = [1 if row_j['CTUID'] in row_i['neighbours'] else 0 for _, row_j in gdf_new.iterrows()]
    master_list.append(adj_list_for_CT_i)

# step 3
W = np.asarray(master_list)

# step 4
row_sums = W.sum(axis=1)
W_std = np.nan_to_num(W / row_sums[:, np.newaxis], 0) #row_sums[:, np.newaxis] reshapes row_sums from (8,) to (8, 1)

# step 5
y = gdf_new['HInc2020'].values  # Array of income values
N = len(y)
y_mean = y.mean()

# Calculate the terms in the Moran's I formula
numerator = sum(sum(W_std[i, j] * (y[i] - y_mean) * (y[j] - y_mean) for j in range(N)) for i in range(N))
denominator = sum((y[i] - y_mean) ** 2 for i in range(N))
morans_I = (N / W_std.sum()) * (numerator / denominator)

# Calculate the terms in the Geary's C formula
numerator_geary = sum(sum(W_std[i, j] * (y[i] - y[j]) ** 2 for j in range(N)) for i in range(N))
gearys_C = ((N - 1) / (2 * W_std.sum())) * (numerator_geary / denominator)

print(f"Global Moran's I: {morans_I}")
print(f"Geary's C: {gearys_C}")

Global Moran's I: -0.5946738022426097
Geary's C: 1.4055523700305812


A negative Moran's I index indiatess tendencies toward dispersion. It means similar values of income are not clustering together. It also agreed with a value of Geary's C because it is greater than 1. Therefore, there is a negative spatial correlation between income in thes areas!To put it diffrently, higher income  areas are adjacent to lower income areas and vice versa.(supporting the presence of income diversity among adjacent areas)


---

## <font color='orange'> Task 2: Calculate the Global Moran's I and Geary's C for the 2020 Income including all Census tracts </font>



*   Show the value of S0
*   Show a few rows of the weights matrix
*   Print the values of the Global Moran's I and the Global Geary's C


<font color='orange'> The following tasks may take a couple minutes to run depending on how you implement them. Using loops it takes me ~2 minutes. Using array multiplication would be faster but not required. </font>

In [ ]:
# step 1  and 2: Create the adjacency matrix for all Census tracts
master_list = []
for i, row_i in gdf_census.iterrows():
    adj_list_for_CT_i = [1 if row_j['CTUID'] in row_i['neighbours'] else 0 for _, row_j in gdf_census.iterrows()]
    master_list.append(adj_list_for_CT_i)

# step 3 (I already have it??)
W = np.asarray(master_list)

# step 4
row_sums = W.sum(axis=1)
#W_std = np.nan_to_num(W / row_sums[:, np.newaxis], 0) #-> warning"invalid value encountered in divide", it does not stop my code running but to remove that:
# Initialize W_std with zeros
W_std = np.zeros_like(W, dtype=float)
# Only normalize rows where the sum is non-zero
non_zero_rows = row_sums > 0  # Boolean array identifying rows with non-zero sums
W_std[non_zero_rows] = W[non_zero_rows] / row_sums[non_zero_rows][:, np.newaxis]

print(f"S0 : {W_std.sum()}")
print(f" a few rows of the weights matrix W:")
print(W)

# step 5
y = gdf_census['HInc2020'].values  # Array of income values
N = len(y)
y_mean = y.mean()

# Calculate the terms in the Moran's I formula
numerator = sum(sum(W_std[i, j] * (y[i] - y_mean) * (y[j] - y_mean) for j in range(N)) for i in range(N))
denominator = sum((y[i] - y_mean) ** 2 for i in range(N))
morans_I_total = (N / W_std.sum()) * (numerator / denominator)

# Calculate the terms in the Geary's C formula
numerator_geary = sum(sum(W_std[i, j] * (y[i] - y[j]) ** 2 for j in range(N)) for i in range(N))
gearys_C_total = ((N - 1) / (2 * W_std.sum())) * (numerator_geary / denominator)

print(f"Global Moran's I for all CT: {morans_I_total}")
print(f"Global Geary's C for all CT: {gearys_C_total}")

S0 : 1217.0
 a few rows of the weights matrix W:
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 1 1 0]]
Global Moran's I for all CT: 0.46544107693089826
Global Geary's C for all CT: 0.5093952454146256


A Global Moran’s I of 0.4654 indicates moderate positive spatial autocorrelation. This suggests that Census Tracts with similar income levels are somewhat clustered together, but the clustering is not very strong. A Geary’s C also shows that similar income levels tend to cluster together (It is near to 0.5, I guess there is some local clustering but also some variation, where not all neighboring areas are highly similar like we got in Task 1!)



---



## <font color='orange'> Task 3: Calculate the Local Moran's I and Geary's C for the 2020 Income  </font>

*   Show the value of S0
*   Show a few rows of the that the sum of the weights matrix
*   Plot *on a map* the Local Moran's I and the Local Geary's C  


In [ ]:
#in calculation of local ones there is no S0? I redo this bc  it has 1 point:)
# Step 1 and 2: Create the adjacency matrix for all Census tracts
master_list = []
for i, row_i in gdf_census.iterrows():
    adj_list_for_CT_i = [1 if row_j['CTUID'] in row_i['neighbours'] else 0 for _, row_j in gdf_census.iterrows()]
    master_list.append(adj_list_for_CT_i)

# Step 3
W = np.asarray(master_list)

# Step 4: Normalize W to create W_std
row_sums = W.sum(axis=1)

#W_std = np.nan_to_num(W / row_sums[:, np.newaxis], 0)  #-> I got a warning: "invalid value encountered in divide". to avoid that:
# Initialize W_std with zeros
W_std = np.zeros_like(W, dtype=float)
# Only normalize rows where the sum is non-zero
non_zero_rows = row_sums > 0  # Boolean array identifying rows with non-zero sums
W_std[non_zero_rows] = W[non_zero_rows] / row_sums[non_zero_rows][:, np.newaxis]

S0 = W_std.sum()  # Sum of all weights in W
print(f"S0: {S0}")
print("A few rows of the weights matrix W:")
print(W)


# Step 5: Prepare variables for income and calculate Local Moran's I and Geary's C
y = gdf_census['HInc2020'].values  # Array of income values
N = len(y)
y_mean = y.mean()

# Calculate Local Moran's I for each Census tract
local_morans_I = []
for i in range(N):
    # Local Moran's I for observation i
    Ii = (y[i] - y_mean) * sum(W_std[i, j] * (y[j] - y_mean) for j in range(N))
    local_morans_I.append(Ii)

# Calculate Local Geary's C for each Census tract
local_gearys_C = []
for i in range(N):
    # Local Geary's C for observation i
    Ci = sum(W_std[i, j] * (y[i] - y[j]) ** 2 for j in range(N))
    local_gearys_C.append(Ci)

# Add Local Moran's I and Local Geary's C to gdf_census
gdf_census['Local_Morans_I'] = local_morans_I
gdf_census['Local_Gearys_C'] = local_gearys_C

# Plot Local Moran's I
fig_morans = px.choropleth_mapbox(
    gdf_census,
    geojson=gdf_census.geometry,
    locations=gdf_census.index,
    color="Local_Morans_I",
    labels={'Local_Morans_I': "Local Moran's I"},
    color_continuous_scale="Sunset",
    center=dict(lat=gdf_census.geometry.centroid.y.mean(), lon=gdf_census.geometry.centroid.x.mean()),
    mapbox_style="open-street-map",
    opacity=0.9,
    zoom=8, #it was 11
    height=600,
    width=800
)
fig_morans.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_morans.show()

# Plot Local Geary's C
fig_gearys = px.choropleth_mapbox(
    gdf_census,
    geojson=gdf_census.geometry,
    locations=gdf_census.index,
    color="Local_Gearys_C",
    labels={'Local_Gearys_C': "Local Geary's C"},
    color_continuous_scale="Sunset",
    center=dict(lat=gdf_census.geometry.centroid.y.mean(), lon=gdf_census.geometry.centroid.x.mean()),
    mapbox_style="open-street-map",
    opacity=0.9,
    zoom=8, #it was 11
    height=600,
    width=800
)
fig_gearys.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_gearys.show()

# Verify that the sum of Local Moran's I is equal to gamma * Global Moran's I
denominator = sum((y[i] - y_mean) ** 2 for i in range(N)) # it is actually numerator :)
gamma_moran = (S0 * denominator) / N
sum_local_morans_I = sum(local_morans_I)
print(f"Sum of Local Moran's I: {sum_local_morans_I}")
print(f"gamma * Global Moran's I: {gamma_moran * morans_I_total}")

# Verify for Geary's C as well
gamma_geary = ((2 * S0)/(N - 1)) * denominator
sum_local_gearys_C = sum(local_gearys_C)
print(f"Sum of Local Geary's C: {sum_local_gearys_C}")
print(f"gamma * Global Geary's C: {gamma_geary * gearys_C_total}")
# I got a warning, try to slove it but I was not succesful!

Output hidden; open in https://colab.research.google.com to view.




---



## <font color='orange'> Optional Task: Calculate the Local Moran's I and Geary's C for a variable with strong correlation pattern </font>

<font color='orange'> Real-world data do not always show strong spatial patterns, as we saw in T3. Here, I am creating a new data 'ForcedCorrel' in which the spatial pattern is controlled by three areas of strong correlation. You can use the same code from T3 to see how the LISAs help (or not) identify these spots. </font>

the solution would be the same as Task 3. It is only there to show you how the LISAs behave in data with a stronger spatial pattern.

In [ ]:
# Here I am creating some synthetic data with strong correlation at three locations
v = []
for i in range(gdf_census.shape[0]):

  value = 200000
  if gdf_census.loc[i,'geometry'].centroid.distance(gdf_census.loc[726,'geometry'].centroid) < 0.1:
     value = 250000
  elif gdf_census.loc[i,'geometry'].centroid.distance(gdf_census.loc[635,'geometry'].centroid) < 0.1:
     value = 150000

  elif gdf_census.loc[i,'geometry'].centroid.distance(gdf_census.loc[9,'geometry'].centroid) < 0.1:
     value = 100000
  v.append(value)

gdf_census['ForcedCorrel'] = v
mean_y = gdf_census['ForcedCorrel'].mean()

# Calculate the sum of weights
S0 = np.sum(W_std)

# Get the number of data points
N = gdf_census.shape[0]

f'Mean for 2020 income {mean_y}, sum of weights {S0}, and number of data points {N}.'

'Mean for 2020 income 194458.12807881774, sum of weights 1217.0, and number of data points 1218.'

In [ ]:
# Plot
fig = px.choropleth_mapbox(
    gdf_census,
    geojson=gdf_census.geometry,
    locations=gdf_census.index,
    color="ForcedCorrel",
    color_continuous_scale="Sunset",
    center=dict(lat=gdf_new.loc[0,'geometry'].centroid.y, lon=gdf_new.loc[0,'geometry'].centroid.x),
    mapbox_style="open-street-map",
    opacity=0.9,
    zoom=8,
    height=600,
    width=800
)
fig.update_layout(margin={"r":1,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Calculate and plot on a map the LISAs for this variable 'ForcedCorrel'



---



---

